## Init

In [1]:
import sys
# print(sys.path)

!python --version

!pip install -q mediapipe

# 모델 파일 다운로드 (Pose Landmarker Lite, Float16, v1)
!wget -O pose_landmarker.task -q https://storage.googleapis.com/mediapipe-models/pose_landmarker/pose_landmarker_heavy/float16/1/pose_landmarker_heavy.task

Python 3.12.7


'wget'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.


## 실시간으로 MPP 동작

### 실시간

- 왼쪽-오른쪽 구분 로직 추가
- 30초 기준으로 중단

In [ ]:
import math
import time
import cv2
import numpy as np
import mediapipe as mp

# MediaPipe 초기화
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

# 스쿼트 영상 경로
# VIDEO_PATH = 'C:\\Users\\mia00\\Desktop\\CD\\resources\\squat.mp4'
MAX_WIDTH = 640  # 영상 최대 너비
TIME_LIMIT = 10  # 시간 제한 (초)
BACK_STRAIGHTNESS_THRESHOLD = 0.15  # 허리 펴짐 기준
SIDE_VIS_MARGIN = 0.15          # visibility 합 차이 최소 마진
SIDE_STABILITY_FRAMES = 8       # 연속 프레임 유지 필요 수
MIN_VIS = 0.7 # 최소 visibility
SESSION_MAX_SECONDS = 30 # 30초
session_running = False
session_start_time = None
last_display_frame = None

SIDE_IDX = {
    "left": {
        "HIP": mp_pose.PoseLandmark.LEFT_HIP.value,
        "KNEE": mp_pose.PoseLandmark.LEFT_KNEE.value,
        "ANKLE": mp_pose.PoseLandmark.LEFT_ANKLE.value,
        "SHOULDER": mp_pose.PoseLandmark.LEFT_SHOULDER.value
    },
    "right": {
        "HIP": mp_pose.PoseLandmark.RIGHT_HIP.value,
        "KNEE": mp_pose.PoseLandmark.RIGHT_KNEE.value,
        "ANKLE": mp_pose.PoseLandmark.RIGHT_ANKLE.value,
        "SHOULDER": mp_pose.PoseLandmark.RIGHT_SHOULDER.value
    }
}

In [ ]:
def joints_visible(landmarks, side, min_vis=MIN_VIS):
    ids = SIDE_IDX[side]
    needed = ["HIP", "KNEE", "ANKLE", "SHOULDER"]
    vis_map = {name: landmarks[ids[name]].visibility for name in needed}
    ok = all(vis_map[n] >= min_vis for n in needed)
    return ok, vis_map

In [ ]:
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)

    ab = a - b
    bc = c - b

    cosine_angle = np.dot(ab, bc) / (np.linalg.norm(ab) * np.linalg.norm(bc))
    angle = np.arccos(np.clip(cosine_angle, -1.0, 1.0))
    return np.degrees(angle)

In [ ]:
def check_side(landmarks, prev_side, stability_counter):
    l_ids = SIDE_IDX["left"]
    r_ids = SIDE_IDX["right"]
    l_sum = (landmarks[l_ids["HIP"]].visibility +
             landmarks[l_ids["KNEE"]].visibility +
             landmarks[l_ids["ANKLE"]].visibility)
    r_sum = (landmarks[r_ids["HIP"]].visibility +
             landmarks[r_ids["KNEE"]].visibility +
             landmarks[r_ids["ANKLE"]].visibility)

    candidate = prev_side
    diff = l_sum - r_sum
    if prev_side is None:
        if l_sum >= r_sum:
            candidate = "left"
        else:
            candidate = "right"
        stability_counter = 0
    else:
        if diff > SIDE_VIS_MARGIN and prev_side != "left":
            candidate = "left"
        elif diff < -SIDE_VIS_MARGIN and prev_side != "right":
            candidate = "right"
        else:
            return prev_side, stability_counter, l_sum, r_sum

        if candidate != prev_side:
            stability_counter += 1
            if stability_counter >= SIDE_STABILITY_FRAMES:
                prev_side = candidate
                stability_counter = 0
            else:
                candidate = prev_side
        else:
            stability_counter = 0

    return candidate, stability_counter, l_sum, r_sum

In [ ]:
def check_knee_angle(landmarks, side):
    ids = SIDE_IDX[side]

    hip = [landmarks[ids["HIP"]].x, landmarks[ids["HIP"]].y]
    knee = [landmarks[ids["KNEE"]].x, landmarks[ids["KNEE"]].y]
    ankle = [landmarks[ids["ANKLE"]].x, landmarks[ids["ANKLE"]].y]

    # 무릎 각도 계산
    knee_angle = calculate_angle(hip, knee, ankle)

    # 피드백 생성
    if knee_angle < 90:
        feedback = f"Good! Knee angle: {knee_angle:.2f} degrees"
        # is_bent_correctly = True
    else:
        feedback = f"Bad! Knee angle: {knee_angle:.2f}degrees"
        # is_bent_correctly = False

    # return feedback, is_bent_correctly
    return feedback, knee_angle

In [ ]:
def draw_feedback(image, feedback, state, squat_count, extra_lines=None):
    # 스쿼트 상태 및 횟수 표시
    cv2.putText(
        image,
        f"State: {state}, Squat Count: {squat_count}",
        # (50, 50),  # 텍스트 위치 (x, y)
        # cv2.FONT_HERSHEY_SIMPLEX,
        # 1.0, (0, 255, 0), 2, cv2.LINE_AA
        (30, 50),
        cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,255,0), 2, cv2.LINE_AA
    )

    # 피드백 정보 표시 (우측 상단)
    cv2.putText(
        image,
        feedback,
        (30, 90),
        cv2.FONT_HERSHEY_SIMPLEX,
        0.8, (255,0,0), 2, cv2.LINE_AA
    )

    if extra_lines:
        y = 130
        for line in extra_lines:
            cv2.putText(image, line, (30, y),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        0.6, (0, 255, 255), 2, cv2.LINE_AA)
            y += 35

In [ ]:
def resize_image(image, max_width):
    w = image.shape[1]
    if w <= max_width:
        return image
    scale = max_width / w
    h = int(image.shape[0] * scale)
    return cv2.resize(image, (max_width, h))

In [ ]:
def process_state(landmarks, side, state, initial_hip_y_dict, sit_start_time, squat_count):
    ids = SIDE_IDX[side]
    hip_y = landmarks[ids["HIP"]].y
    knee_y = landmarks[ids["KNEE"]].y
    shoulder_y = landmarks[ids["SHOULDER"]].y

    # 처음 자세의 엉덩이 위치 저장
    if initial_hip_y_dict[side] is None:
        initial_hip_y_dict[side] = hip_y
        print(f"Initial Hip Y: {hip_y:.3f}")
    init_hip = initial_hip_y_dict[side]

    visible_ok, vis_map = joints_visible(landmarks, side)
    if not visible_ok:
        # 스쿼트 판정/카운트 건너뜀
        feedback = "Low visibility - hold still"
        extra = [f"{k}:{vis_map[k]:.2f}" for k in vis_map]  # 예: HIP:0.63 ...
        return state, squat_count, feedback, sit_start_time, initial_hip_y_dict, extra

    # 허리 펴짐 정도 계산
    back_straightness = abs(hip_y - shoulder_y)

    feedback = ""
    extra = []

    # 상태 전환 로직
    if state == "START":
        feedback = "Check posture..."
        if back_straightness < BACK_STRAIGHTNESS_THRESHOLD:
            extra.append("Back: Good")
        else:
            extra.append("Back: Straighten!")
        # 엉덩이 :=: 무릎
        if abs(hip_y - knee_y) < 0.05:
            state = "SIT"
            sit_start_time = time.time()
            print("State -> SIT")
    elif state == "SIT":
        feedback, knee_angle = check_knee_angle(landmarks, side)
        extra.append(f"KneeAngle: {knee_angle:.1f}")

        if time.time() - sit_start_time > TIME_LIMIT:
            state = "START"
            feedback = "Timeout. Reset."
            print("Timeout -> START")
        # 힙이 무릎보다 위 + 초기 힙 위치와 비슷
        elif hip_y < knee_y - 0.05 and (hip_y <= init_hip + 0.05):
            state = "STAND"
            squat_count += 1
            feedback = f"Squat Count: {squat_count}"
            print(f"스쿼트 횟수: {squat_count}")
    elif state == "STAND":
        if back_straightness < BACK_STRAIGHTNESS_THRESHOLD:
            feedback = "Standing: Good posture"
        else:
            feedback = "Standing: Straighten back"
        if hip_y >= knee_y - 0.05:
            state = "SIT"
            sit_start_time = time.time()
            feedback = "Transitioning to SIT."
            print("State -> SIT")

    return state, squat_count, feedback, sit_start_time, initial_hip_y_dict, extra

In [ ]:
cap = cv2.VideoCapture(0) # 실시간 웹캠 사용

# 전역 변수 초기화
squat_count = 0
initial_hip_y_dict = {"left": None, "right": None}
state = "START"
sit_start_time = None

active_side = None
side_stability_counter = 0

# FPS 측정용
prev_time = time.time()
frame_count = 0
fps = 0.0

# 세션 변수
session_running = False
session_start_time = None
last_display_frame = None

with mp_pose.Pose(
    model_complexity=1,  # 0: lite, 1: full, 2: heavy
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as pose:
  while cap.isOpened():
    if not session_running and last_display_frame is not None:
      freeze = last_display_frame.copy()
      cv2.imshow("MediaPipe Pose", resize_image(freeze, MAX_WIDTH))
      key = cv2.waitKey(1) & 0xFF
      if key in (27, ord('q')):
        break
      continue

    ret, frame = cap.read()
    if not ret:
      print("Frame grab failed.")
      break

    # BGR → RGB 변환
    frame.flags.writeable = False
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(rgb)
    feedback = ""  # 피드백 초기화
    extra_lines = []

    if session_running and results.pose_landmarks:
      landmarks = results.pose_landmarks.landmark
      active_side, side_stability_counter, left_sum, right_sum = check_side(
        landmarks, active_side, side_stability_counter
      )
      if active_side is not None:
        state, squat_count, feedback, sit_start_time, initial_hip_y_dict, extra_lines = process_state(landmarks, active_side, state, initial_hip_y_dict, sit_start_time, squat_count)
    elif not session_running:
      feedback = "stopped"

    frame.flags.writeable = True
    frame = cv2.cvtColor(rgb, cv2.COLOR_RGB2BGR)
    if results.pose_landmarks:
            mp_drawing.draw_landmarks(
                frame,
                results.pose_landmarks,
                mp_pose.POSE_CONNECTIONS,
                landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style()
    )

    if session_running and session_start_time:
      elapsed = time.time() - session_start_time
      cv2.putText(frame, f"Elapsed: {elapsed:5.1f}s / {SESSION_MAX_SECONDS}s",
                  (300, 25), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,255,0), 2,
                  cv2.LINE_AA)
      if elapsed >= SESSION_MAX_SECONDS:
        session_running = False
        last_display_frame = frame.copy()

    # 피드백 및 상태 정보 표시
    draw_feedback(frame, feedback, state, squat_count, extra_lines)

    # FPS 계산 (0.5초마다 갱신)
    frame_count += 1
    now = time.time()
    if now - prev_time >= 0.5:
      fps = frame_count / (now - prev_time)
      prev_time = now
      frame_count = 0
    cv2.putText(frame, f"FPS: {fps:.1f}", (30, frame.shape[0]-20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2, cv2.LINE_AA)

    if not session_running:
      cv2.putText(frame, "Press 's' to START (30s session)", (30, frame.shape[0]-55),
                  cv2.FONT_HERSHEY_SIMPLEX, 0.6, (200,200,200), 2, cv2.LINE_AA)
    elif session_running:
      cv2.putText(frame, "Press 'e' to END",
                  (30, frame.shape[0]-55),
                  cv2.FONT_HERSHEY_SIMPLEX, 0.6, (200,200,200), 2, cv2.LINE_AA)

    # 영상 크기 조정 (원본 비율 유지, 최대 너비 720px로 제한)
    resized_image = resize_image(frame, MAX_WIDTH)

    if session_running:
      last_display_frame = frame.copy()

    # Flip 제거 (원본 영상 그대로 표시)
    cv2.imshow('MediaPipe Pose', resized_image)
    key = cv2.waitKey(1) & 0xFF
    if key in (27, ord('q')):
      break
    if key == ord('s') and not session_running:
      session_running = True
      session_start_time = time.time()
    if key == ord('e') and session_running:
      session_running = False
      last_display_frame = frame.copy()

cap.release()
cv2.destroyAllWindows()

Initial Hip Y: 1.436
Initial Hip Y: 1.055


### 실시간

- 왼쪽-오른쪽 구분 로직 추가
- 스쿼트 횟수로 중단

[x] 횟수 따른 결과 출력

[] 각 포지션에 따른 정확도 계산

    - 각 포지션의 실수 퍼센트 계산 후 출력

[] 자세가 안 좋아도 카운트는 하기

[] 처음 시작 시 시작함을 알리기

[] 화면 크기 키우기

[] 폰트 색 잘 보이는 색으로 통일하기

[x] 내장 카메라 / 캠
- 내장 카메라 0
- 외장 USB 웹캠 1

In [42]:
import math
import time
import cv2
import numpy as np
import mediapipe as mp

# MediaPipe 초기화
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

# 스쿼트 영상 경로
# CAM_INDEX = 0  # 내장 카메라
CAM_INDEX = 1 # 외장 USB 웹캠

USE_DIRECTSHOW = True
RESTART_RESET = False

MAX_WIDTH = 640  # 영상 최대 너비
TIME_LIMIT = 10  # 시간 제한 (초)
BACK_STRAIGHTNESS_THRESHOLD = 0.15  # 허리 펴짐 기준
SIDE_VIS_MARGIN = 0.15          # visibility 합 차이 최소 마진
SIDE_STABILITY_FRAMES = 8       # 연속 프레임 유지 필요 수
MIN_VIS = 0.7 # 최소 visibility
# SESSION_MAX_SECONDS = 30 # 30초
session_running = False
session_start_time = None
last_display_frame = None
TARGET_REPS = 2 # 목표 스쿼트 반복 횟수
TARGET_REP_HOLD_FRAMES = 5

SIDE_IDX = {
    "left": {
        "HIP": mp_pose.PoseLandmark.LEFT_HIP.value,
        "KNEE": mp_pose.PoseLandmark.LEFT_KNEE.value,
        "ANKLE": mp_pose.PoseLandmark.LEFT_ANKLE.value,
        "SHOULDER": mp_pose.PoseLandmark.LEFT_SHOULDER.value
    },
    "right": {
        "HIP": mp_pose.PoseLandmark.RIGHT_HIP.value,
        "KNEE": mp_pose.PoseLandmark.RIGHT_KNEE.value,
        "ANKLE": mp_pose.PoseLandmark.RIGHT_ANKLE.value,
        "SHOULDER": mp_pose.PoseLandmark.RIGHT_SHOULDER.value
    }
}

In [26]:
def joints_visible(landmarks, side, min_vis=MIN_VIS):
    ids = SIDE_IDX[side]
    needed = ["HIP", "KNEE", "ANKLE", "SHOULDER"]
    vis_map = {name: landmarks[ids[name]].visibility for name in needed}
    ok = all(vis_map[n] >= min_vis for n in needed)
    return ok, vis_map

In [27]:
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)

    ab = a - b
    bc = c - b

    cosine_angle = np.dot(ab, bc) / (np.linalg.norm(ab) * np.linalg.norm(bc))
    angle = np.arccos(np.clip(cosine_angle, -1.0, 1.0))
    return np.degrees(angle)

In [28]:
def check_side(landmarks, prev_side, stability_counter):
    l_ids = SIDE_IDX["left"]
    r_ids = SIDE_IDX["right"]
    l_sum = (landmarks[l_ids["HIP"]].visibility +
             landmarks[l_ids["KNEE"]].visibility +
             landmarks[l_ids["ANKLE"]].visibility)
    r_sum = (landmarks[r_ids["HIP"]].visibility +
             landmarks[r_ids["KNEE"]].visibility +
             landmarks[r_ids["ANKLE"]].visibility)

    candidate = prev_side
    diff = l_sum - r_sum
    if prev_side is None:
        if l_sum >= r_sum:
            candidate = "left"
        else:
            candidate = "right"
        stability_counter = 0
    else:
        if diff > SIDE_VIS_MARGIN and prev_side != "left":
            candidate = "left"
        elif diff < -SIDE_VIS_MARGIN and prev_side != "right":
            candidate = "right"
        else:
            return prev_side, stability_counter, l_sum, r_sum

        if candidate != prev_side:
            stability_counter += 1
            if stability_counter >= SIDE_STABILITY_FRAMES:
                prev_side = candidate
                stability_counter = 0
            else:
                candidate = prev_side
        else:
            stability_counter = 0

    return candidate, stability_counter, l_sum, r_sum

In [29]:
def check_knee_angle(landmarks, side):
    ids = SIDE_IDX[side]

    hip = [landmarks[ids["HIP"]].x, landmarks[ids["HIP"]].y]
    knee = [landmarks[ids["KNEE"]].x, landmarks[ids["KNEE"]].y]
    ankle = [landmarks[ids["ANKLE"]].x, landmarks[ids["ANKLE"]].y]

    # 무릎 각도 계산
    knee_angle = calculate_angle(hip, knee, ankle)

    # 피드백 생성
    if knee_angle < 90:
        feedback = f"Good! Knee angle: {knee_angle:.2f} degrees"
        # is_bent_correctly = True
    else:
        feedback = f"Bad! Knee angle: {knee_angle:.2f}degrees"
        # is_bent_correctly = False

    # return feedback, is_bent_correctly
    return feedback, knee_angle

In [30]:
def draw_feedback(image, feedback, state, squat_count, extra_lines=None):
    # 스쿼트 상태 및 횟수 표시
    cv2.putText(
        image,
        f"State: {state}, Squat Count: {squat_count}",
        # (50, 50),  # 텍스트 위치 (x, y)
        # cv2.FONT_HERSHEY_SIMPLEX,
        # 1.0, (0, 255, 0), 2, cv2.LINE_AA
        (30, 50),
        cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,255,0), 2, cv2.LINE_AA
    )

    # 피드백 정보 표시 (우측 상단)
    cv2.putText(
        image,
        feedback,
        (30, 90),
        cv2.FONT_HERSHEY_SIMPLEX,
        0.8, (255,0,0), 2, cv2.LINE_AA
    )

    if extra_lines:
        y = 130
        for line in extra_lines:
            cv2.putText(image, line, (30, y),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        0.6, (0, 255, 255), 2, cv2.LINE_AA)
            y += 35

In [31]:
def resize_image(image, max_width):
    w = image.shape[1]
    if w <= max_width:
        return image
    scale = max_width / w
    h = int(image.shape[0] * scale)
    return cv2.resize(image, (max_width, h))

In [32]:
def process_state(landmarks, side, state, initial_hip_y_dict, sit_start_time, squat_count):
    ids = SIDE_IDX[side]
    hip_y = landmarks[ids["HIP"]].y
    knee_y = landmarks[ids["KNEE"]].y
    shoulder_y = landmarks[ids["SHOULDER"]].y

    # 처음 자세의 엉덩이 위치 저장
    if initial_hip_y_dict[side] is None:
        initial_hip_y_dict[side] = hip_y
        print(f"Initial Hip Y: {hip_y:.3f}")
    init_hip = initial_hip_y_dict[side]

    visible_ok, vis_map = joints_visible(landmarks, side)
    if not visible_ok:
        # 스쿼트 판정/카운트 건너뜀
        feedback = "Low visibility - hold still"
        extra = [f"{k}:{vis_map[k]:.2f}" for k in vis_map]  # 예: HIP:0.63 ...
        return state, squat_count, feedback, sit_start_time, initial_hip_y_dict, extra

    # 허리 펴짐 정도 계산
    back_straightness = abs(hip_y - shoulder_y)

    feedback = ""
    extra = []

    # 상태 전환 로직
    if state == "START":
        feedback = "Check posture..."
        if back_straightness < BACK_STRAIGHTNESS_THRESHOLD:
            extra.append("Back: Good")
        else:
            extra.append("Back: Straighten!")
        # 엉덩이 :=: 무릎
        if abs(hip_y - knee_y) < 0.05:
            state = "SIT"
            sit_start_time = time.time()
            print("State -> SIT")
    elif state == "SIT":
        feedback, knee_angle = check_knee_angle(landmarks, side)
        extra.append(f"KneeAngle: {knee_angle:.1f}")

        if time.time() - sit_start_time > TIME_LIMIT:
            state = "START"
            feedback = "Timeout. Reset."
            print("Timeout -> START")
        # 힙이 무릎보다 위 + 초기 힙 위치와 비슷
        elif hip_y < knee_y - 0.05 and (hip_y <= init_hip + 0.05):
            state = "STAND"
            squat_count += 1
            feedback = f"Squat Count: {squat_count}"
            print(f"스쿼트 횟수: {squat_count}")
    elif state == "STAND":
        if back_straightness < BACK_STRAIGHTNESS_THRESHOLD:
            feedback = "Standing: Good posture"
        else:
            feedback = "Standing: Straighten back"
        if hip_y >= knee_y - 0.05:
            state = "SIT"
            sit_start_time = time.time()
            feedback = "Transitioning to SIT."
            print("State -> SIT")

    return state, squat_count, feedback, sit_start_time, initial_hip_y_dict, extra

In [43]:
cap = cv2.VideoCapture(CAM_INDEX, cv2.CAP_DSHOW if USE_DIRECTSHOW else 0)

# 전역 변수 초기화
squat_count = 0
initial_hip_y_dict = {"left": None, "right": None}
state = "START"
sit_start_time = None

active_side = None
side_stability_counter = 0

# FPS 측정용
prev_time = time.time()
frame_count = 0
fps = 0.0

# 세션 변수
session_running = False
session_start_time = None
last_display_frame = None

# squat goal
target_reached = False
stand_hold = 0

with mp_pose.Pose(
    model_complexity=1,  # 0: lite, 1: full, 2: heavy
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as pose:
  while cap.isOpened():
    if not session_running and last_display_frame is not None:
      freeze = last_display_frame.copy()
      if target_reached:
        cv2.putText(freeze, f"Completed {TARGET_REPS} Reps!",
                    (30, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,0,255), 2, cv2.LINE_AA)
      cv2.putText(freeze, "s : restart, q : quit",
                  (30, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,255), 2, cv2.LINE_AA)
      cv2.imshow("MediaPipe Pose", resize_image(freeze, MAX_WIDTH))

      key = cv2.waitKey(1) & 0xFF
      if key in (27, ord('q')):
        break
      if key == ord('s'):
        state = "START"
        session_running = True
        session_start_time = time.time()
        stand_hold = 0
        print("[Session] RESTART")
      continue

    ret, frame = cap.read()
    if not ret:
      print("Frame grab failed.")
      break

    # BGR → RGB 변환
    frame.flags.writeable = False
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(rgb)
    feedback = ""  # 피드백 초기화
    extra_lines = []

    if session_running and results.pose_landmarks:
      landmarks = results.pose_landmarks.landmark
      active_side, side_stability_counter, left_sum, right_sum = check_side(
        landmarks, active_side, side_stability_counter
      )
      if active_side is not None:
        state, squat_count, feedback, sit_start_time, initial_hip_y_dict, extra_lines = process_state(landmarks, active_side, state, initial_hip_y_dict, sit_start_time, squat_count)

      if squat_count >= TARGET_REPS:
        if not target_reached:
          target_reached = True
          stand_hold = 0
        if state == "STAND":
          stand_hold += 1
        else:
          stand_hold = 0
        extra_lines.append(f"FinishHold: {stand_hold}/{TARGET_REP_HOLD_FRAMES}")
        feedback = f"Target Reps ({TARGET_REPS}) achieved!"
        if stand_hold >= TARGET_REP_HOLD_FRAMES:
          session_running = False
          last_display_frame = frame.copy()
    elif not session_running:
      feedback = "stopped"

    frame.flags.writeable = True
    frame = cv2.cvtColor(rgb, cv2.COLOR_RGB2BGR)
    if results.pose_landmarks:
            mp_drawing.draw_landmarks(
                frame,
                results.pose_landmarks,
                mp_pose.POSE_CONNECTIONS,
                landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style()
    )

    # 피드백 및 상태 정보 표시
    draw_feedback(frame, feedback, state, squat_count, extra_lines)

    # FPS 계산 (0.5초마다 갱신)
    frame_count += 1
    now = time.time()
    if now - prev_time >= 0.5:
      fps = frame_count / (now - prev_time)
      prev_time = now
      frame_count = 0
    cv2.putText(frame, f"FPS: {fps:.1f}", (30, frame.shape[0]-20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2, cv2.LINE_AA)

    if not session_running:
      cv2.putText(frame, "s : start, goal : {TARGET_REPS}", (30, frame.shape[0]-55),
                  cv2.FONT_HERSHEY_SIMPLEX, 0.6, (200,200,200), 2, cv2.LINE_AA)
    else:
      cv2.putText(frame, "e : quit",
                  (30, frame.shape[0]-55),
                  cv2.FONT_HERSHEY_SIMPLEX, 0.6, (200,200,200), 2, cv2.LINE_AA)

    # 영상 크기 조정 (원본 비율 유지, 최대 너비 720px로 제한)
    resized_image = resize_image(frame, MAX_WIDTH)

    if session_running:
      last_display_frame = frame.copy()

    # Flip 제거 (원본 영상 그대로 표시)
    cv2.imshow('MediaPipe Pose', resized_image)

    key = cv2.waitKey(1) & 0xFF
    if key in (27, ord('q')):
      break
    if key == ord('s') and not session_running:
      session_running = True
      session_start_time = time.time()
    if key == ord('e') and session_running:
      session_running = False
      last_display_frame = frame.copy()

cap.release()
cv2.destroyAllWindows()

Initial Hip Y: 0.237
State -> SIT
Timeout -> START
State -> SIT
Initial Hip Y: 0.363
스쿼트 횟수: 1
State -> SIT
스쿼트 횟수: 2
